In [14]:
from sdid import sdid
import pandas as pd
from get_data import quota, california_prop99

In [15]:
a = sdid(california_prop99(), "State", "Year", "treated", "PacksPerCapita")
print(a)

-15.60383


In [16]:
b = sdid(quota(), "country", "year", "quota", "womparl")
print(b)

8.03402


In [17]:
qta = quota()
drop_country = [
	"Algeria", "Kenya", "Samoa", "Swaziland", "Tanzania"
]
quota_if = qta.query("country not in @drop_country")
c = sdid(quota_if, "country", "year", "quota", "womparl")
c

10.33052

In [1]:
import numpy as np, pandas as pd
from get_data import quota, california_prop99
from utils import panel_matrices, collapse_form, sum_normalize, att_mult
from solver import fw_step, sc_weight_fw, sc_weight_covariates
from sdid import sparsify_function, varianza, sdid
# sdid(quota(), "country", "year", "quota", "womparl", ["lnmmrt", "lngdp"])

In [3]:
cal1 = sdid(california_prop99(), "State", "Year", "treated", "PacksPerCapita")
cal1["att"]

-15.60383

## Vcov

### Jackknife

In [7]:
quota_data = quota()
not_in_unit = ["Algeria", "Kenya", "Samoa", "Swaziland", "Tanzania"]
# quota_data1 = quota_data[~quota_data["country"].isin(not_in_unit)].dropna(subset=["country"])
# quota_data1.info()
quota_drop = quota_data.query("country not in @not_in_unit")
q1 = sdid(quota_drop, "country", "year", "quota", "womparl")
q1

{'att': 10.33066,
 'att_info':      time   att_time    att_wt   N0  T0  N1  T1
 0  2002.0   6.967746  0.518519  110  12   2  14
 1  2003.0  13.952256  0.481481  110  13   2  13,
 'weights': {'lambda': [array([0.        , 0.18692424, 0.        , 0.        , 0.        ,
          0.        , 0.        , 0.        , 0.11309248, 0.12219293,
          0.        , 0.57779035]),
   array([0.        , 0.18422749, 0.        , 0.        , 0.        ,
          0.        , 0.        , 0.        , 0.0887519 , 0.05663246,
          0.        , 0.        , 0.67038816])],
  'omega': [array([0.00686606, 0.00308393, 0.        , 0.00401   , 0.00134109,
          0.01125434, 0.00545765, 0.00999792, 0.        , 0.00923891,
          0.01667525, 0.01343713, 0.        , 0.0141093 , 0.        ,
          0.00976986, 0.02344187, 0.01092555, 0.00921574, 0.00893749,
          0.01470498, 0.        , 0.00544846, 0.01022   , 0.00497425,
          0.007856  , 0.00722675, 0.00858603, 0.        , 0.01078756,
       

In [9]:
estimate = q1
data_ref, time_break = estimate["data_ref"], estimate["break_points"]

In [10]:
uniqID = np.unique(data_ref["unit"])
N = len(uniqID)
weigths = estimate["weights"]
lambda_estimate, omega_estimate = np.array(weigths["lambda"], dtype = object), np.array(weigths["omega"], dtype = object)

In [26]:
ind, id_id = 0, 0

def theta_jk(ind, _id):
    omega_aux = sum_normalize(omega_estimate[_id])
    if ind < len(omega_estimate):
        omega_aux = sum_normalize(np.delete(omega_estimate[_id], ind))
    lambda_aux = lambda_estimate[_id]
    drop_unit, tyear_filter = uniqID[ind], [0, time_break[_id]]
    
    data_aux = data_ref.query("unit not in @drop_unit").query("tyear in @tyear_filter")
    yng = data_aux.groupby('unit').size().shape[0]
    ynt = data_aux.groupby('unit').size().min()
    N1 = int((data_aux['tyear'] == time_break[_id]).sum() / ynt)
    npre = data_aux[data_aux['time'] < time_break[_id]].groupby('time').size().shape[0]
    T1 = int(ynt - npre)
    tau_wt_aux = N1 * T1

    Y = data_aux.pivot_table(index="unit", columns="time", values="outcome", sort=False)
    Y_beta = np.array(Y)
    nt = Y.shape
    N1, T1 = int(nt[0] - len(omega_aux)), int(nt[1] - len(lambda_aux))
    weights_omega = np.concatenate((-omega_aux, np.full(N1, 1/N1)))
    weights_lambda = np.concatenate((-lambda_aux, np.full(T1, 1/T1)))
    tau_aux = np.dot(weights_omega, Y).dot(weights_lambda)

    # return pd.DataFrame({'unit': [uniqID[ind]], 'time': [time_break[_id]], 'tau_aux': [tau_aux], 'tau_wt_aux': [tau_wt_aux]})


In [27]:
for i, j in itertools.product(range(N), range(len(time_break))):
    theta_jk(i, j)

In [37]:
theta_jk(25, 1)
import itertools
att_table = pd.concat([theta_jk(i, j) for i, j in itertools.product(range(N), range(len(time_break)))], ignore_index=True)


In [43]:
result = att_table.groupby('unit', group_keys=True)\
    .apply(
        lambda x: x.assign(
            tau_wt=x['tau_wt_aux'] / x['tau_wt_aux'].sum(),
            # att_aux= (x['tau_aux'] * (x['tau_wt_aux'] / x['tau_wt_aux'].sum())).sum()
            att_aux=x['tau_aux'] * (x['tau_wt_aux'] / x['tau_wt_aux'].sum())
            )
        ).reset_index(drop=True)
result

,unit,time,tau_aux,tau_wt_aux,tau_wt,att_aux
0,Albania,2002.0,6.979981,28,0.518519,3.619249
1,Albania,2003.0,13.952256,26,0.481481,6.717753
2,Angola,2002.0,6.987142,28,0.518519,3.622962
3,Angola,2003.0,14.136355,26,0.481481,6.806393
4,Argentina,2002.0,6.959207,28,0.518519,3.608478
...,...,...,...,...,...,...
223,"Yemen, Rep.",2003.0,26.303416,26,0.481481,12.664608
224,Zambia,2002.0,7.197983,28,0.518519,3.732288
225,Zambia,2003.0,26.303416,26,0.481481,12.664608
226,Zimbabwe,2002.0,7.227991,28,0.518519,3.747847


In [49]:
att_aux = result.groupby("unit").sum().att_aux.to_numpy()
((N-1)/N) * (N - 1) * varianza(att_aux)

265.1315201018747

In [58]:
att_aux_all = []
for i in range(N):
	ref_unit = uniqID[i]
	ref_tbl_unit = att_table.query("unit in @ref_unit")
	tau_ref_aux = ref_tbl_unit.tau_aux
	tau_wt_aux = ref_tbl_unit.tau_wt_aux
	tau_wt_aux = tau_wt_aux / np.sum(tau_wt_aux)
	att_aux = np.dot(tau_ref_aux, tau_wt_aux)
	att_aux_all.append(att_aux)

## Bootstrap

In [ ]:
q2 = sdid(quota(), "country", 'year', 'quota', 'womparl')
q2

In [3]:
estimate = q2
data_ref = estimate["data_ref"]
uniqueID = np.unique(data_ref.unit)
N = len(uniqueID)

In [4]:
def theta_bt():
    sample_id = np.random.choice(uniqueID, replace=True, size=N)
    def sample_concat(_id):
        sample_id_n = sample_id[_id]
        data_c = data_ref.query("unit in @sample_id_n")
        data_c = data_c.assign(unit1=str(data_c['unit']) + str(_id))
        return data_c
    sampled_df = pd.concat([sample_concat(i) for i in range(N)], ignore_index=True)
    if len(np.unique(sampled_df.treatment)) != 2:
        theta_bt()
    att_aux = sdid(sampled_df, "unit1", "time", "treatment", "outcome")["att"]
    return att_aux


In [18]:
t = 0
att_bt = np.array([])
n_reps = 50

while t < n_reps:
    t+= 1
    aux = theta_bt()
    att_bt = np.append(att_bt, aux)

In [19]:
se_bootstrap = np.sqrt(1 / n_reps * np.sum((att_bt - np.sum(att_bt / n_reps)) ** 2))
se_bootstrap
# return(se_bootstrap)()

3.743576763233814

##  Placebo

In [26]:
data_ref = estimate["data_ref"]
tr_years = data_ref.query("time == tyear and tyear != 0").time
N_tr = len(tr_years)
df_co = data_ref.query("treated == 0")
units_df_co = np.unique(df_co.unit)
N_co = len(units_df_co)
N_aux = N_co - N_tr

In [88]:
def theta_pb():
	plabeo_years = pd.DataFrame({
		"unit" : np.random.choice(units_df_co, size = N_tr),
		'tyear1' : tr_years
		})
	aux_data = df_co.merge(plabeo_years, on="unit", how='outer').sort_values("tyear1")
	aux_data = aux_data.assign(
		tyear = aux_data.tyear1.fillna(aux_data["tyear"])
	)
	aux_data = aux_data.assign(
		treatment = np.where(((aux_data.tyear != 0) & (aux_data.time == aux_data.tyear)), 1, 0)
		).groupby("unit", group_keys=False).apply(
			lambda x: x.assign(
				treated = x["treatment"].max()
			)
		).reset_index(drop = True)
	att = sdid(aux_data, "unit", "time", "treatment", "outcome")
	return att["att"]


In [94]:
t = 0
att_pb = np.array([])
n_reps = 50

while t < n_reps:
    t += 1
    aux = theta_pb()
    att_pb = np.append(att_pb, aux)


In [98]:
se_placebo = np.sqrt(1 / n_reps * np.sum((att_pb - np.sum(att_pb / n_reps)) ** 2))
se_placebo

1.5591266442311735